## Population
Primero creamos la clase poblacion, tendra atributos generales de toda la poblacion como el nombre, tama;o, ploidia y vida media (habran mas seguramente).

Despues se crea la clase individuo, que hereda de poblacion sus atributos y ademas tendra otros tales como el identificador, la edad, el sexo, su genotipo y quiza sus padres **ver como implementar**

### Funciones
#### init
Inicializa los atributos, que son
- **name** : nombre de la poblacion, no relevante
- **size** : tama;o de la poblacion, utilizado para recorrer posteriormente los individuos entre otras
- **ploidy** : indica si la pob esta formada por individuos haploides(1) o diploides(2)
- **vida media** : todavia sin uso, se implementara posteriormente para generar una dist de edades y para hacerla evolucionar.

Estos son los atributos que el usuario debera pasar (tiene valores por defecto), ademas se encuentran otros como:

- **gen1List,gen2List** : lista con los genotipos(alelos) posibles, ahora mismo esto da problemas porque tienen que ser accesibles por la subclase Individuals pero no lo son.
#### _getGen..
Este nuevo metodo se ha hecho para hacer accesible los atributos fuera de la clase y en la subclase pero tambien da problemas.

#### generateIndividuals
No se le pasa ningun parametro (posiblemente se cambie ya que el usuario deberia poder elegir ciertas cosas), tampoco devuelve ninguno.
Dentro, genera **indiv** que es una lista de objetos tipo **individuo** de tama;o size, ademas printa por pantalla que se ha completado el proceso
#### printIndiv
recorre la lista en un bucle y printa los atributos de cada instancia de individuo en la lista hay que **ver como hacer para printar solo un rango determinado**
#### getMeanAge
Calcula la edad media de la poblacion, de momento siempre 0
#### allGenotypes
Funcion que genera listas globales para que sean accesibles por la clase hija (probablemente haya otra forma mejor de hacerlo), estas listas contienen los alelos posibles para cada gen
#### getGenotype
Todavia no implementada, en un futuro te dira un resumen de los genotipos: frecuencia
#### summary
En un futuro, te dara un resumen de todos los atributos de la poblacion (media de edad, frecuencia de cada sexo, frecuencias alelicas, genotipicas y fenotipicas) todo esto lo hara probablemente llamando a otras funciones


In [44]:
from random import randint

#clase poblacion,atributos generales que heredara de los individuos
class Population:

    def __init__(self,size = 100,
                 name="Homo sapiens",
                 ploidy = 2, 
                 vida_media=55,
                 R=0.1,
                 mu = (1e-4,1e-4),
                 freq={'A':(0.4,0.6),'B':(0.6,0.4)}):
                 
        self.name = name
        self.size = size
        self.ploidy = ploidy
        self.gen1List = []
        self.gen2List = []
        self.vida_media = vida_media
        self.freq = freq
        self.mu = mu
    
    #genera indivividuos
    def generateIndividuals(self):
        #ejecuta la funcion para crear las variables globales que contengan los genotipos posibles
        self.allGenotypes()
        self.indiv = [Individual(i,self.name,self.size,self.ploidy,
                                 self.gen1List,self.gen2List,self.vida_media) 
                      for i in range(self.size)]
        print("se han generado un total de {} individuos de la poblacion {}".format(self.size,self.name))
        
    #printa individuos        
    def printIndiv(self,show=5):
        print("IDENTIF SEX AGE GENOTYPE".replace(" ","\t"))
        for x in self.indiv:
            print (x.ide,x.sex,x.age,*x.genotype.values(),sep="\t")
            #contador inverso, si se han enseñado show elementos para la ejecucion
            show += -1
            if show == 0:
                break
                
    #funcion que calcula las frecuencias genotipicas
    def genotFreq(self):
        self.geneticPool = dict()
        if self.ploidy == 2:
            for key,lista in self.freq.items():
                self.geneticPool[key] = (int(lista[0]**2),
                                         int(lista[0]*lista[1]*2),int(lista[1]**2))
                self.geneticPool[key] = [x*self.size for x in self.geneticPool[key]]
            print(self.geneticPool)
        elif self.ploidy == 1:
            self.geneticPool = self.freq
             
    def getMeanAge(self):
        #obtienes la edad media recorriendo la lista de individuos
        try:
            meanAge = 0
            for obj in self.indiv:
                meanAge += obj.age
            meanAge = round(meanAge/len(self.indiv),2)
            print("la edad media de la poblacion es: ",meanAge)
        except:
            print("No has inicializado la poblacion")
    #esto quedara obsoleto       
    def allGenotypes(self):
       #genera una lista de tuplas(n=ploidy=2), cada tupla contiene un alelo
        self.gen1List = [x for x in itertools.product('Aa',repeat=self.ploidy)]
        #convierte las tuplas en un string ej: ('A','a') to Aa
        self.gen1List = [''.join(elem) for elem in self.gen1List]
        #cambia el aA por Aa
        for pos,elemento in enumerate(self.gen1List):
            if elemento == 'aA':
                self.gen1List[pos]='Aa'

        self.gen2List = [x for x in itertools.product('Bb',repeat=self.ploidy)]
        self.gen2List = [''.join(elem) for elem in self.gen2List]
        for pos,elemento in enumerate(self.gen2List):
            if elemento == 'bB':
                self.gen2List[pos]='Bb' 
                
    #esto quedara obsoleto
    def getGenotype(self):
        counter = {'AA':0,'Aa':0,'aa':0,'BB':0,'Bb':0,'bb':0}      
        for elemento in self.indiv:
            for key in counter:
                if elemento.genotype['gen1'] == key or elemento.genotype['gen2']==key:
                    counter[key] += 1
        print(counter)              
    def summary(self):
        pass


# Individual
Individual se trata de una clase que se crea de forma indirecta, llamando un metodo de Population, esta clase no es accesible al usuario. 
Esta clase consta de los atributos y metodos necesarios para dar lugar a un individuo que formara parte de la poblacion, los atributos son: 

## atributos
- Atributos pasados de la clase Population +...
- Sex: tipo string male and female
- Ide: identificador, para cada individuo (mirar de construir la lista de individuos con forma de diccionario para acceder por su identificador)
- Age: numeric int, comienza en 0
- genotype: quiza un diccionario tipo {gen1:'AA',gen2:'Bb'}, en cualquier caso se puede hacer un conjunto con todos los posibles genotipos y escoger aleatoriamente.

## Metodos

### init
Inicializa los atributos vistos anteriormente, la forma de inicializarlos puede no ser la correcta ya que en muchos se esta llamando a un metodo de la clase pero de momento funciona

### sex
Le da un valor de sexo a cada individuo de forma aleatoria al 50%, no requiere ningun parametro, devuelve male o female
### edad
Al igual que sex, asigna una edad a cada individuo. Esto habra que repensarlo porque la edad comienza en 0 para todos, dependiendo de como lo quiera implementar cuando la poblacion empiece a evolucionar esta edad debera ir aumentando para todos, algunos individuos iran muriendo = eliminando de la lista de objetos en la poblacion. Si este metodo hace correr el tiempo quiza seria mejor tenerlo en poblacion ya que el tiempo corre para todos igual *revisar*
### firstGenotipo y chooseGenotype
Genera un diccionario gen:alelos para 2 genes en total (mirar de aumentarlo mas adelante), los alelos se obtienen de la lista de alelos posibles de la clase population y se asigna uno de estos aleatoriamente si asi se quiere, si no se puede inicializar a todos los individuos con el mismo genotipo Aa/Bb. *ver como el usuario pueda pasar esta forma de generar genotipo*, seguramente desde generate individuals se deba pasar un parametro


In [40]:
#clase individuo, ya no hereda de poblacion
import itertools
import random

class Individual():

    def __init__(self,nom,name,size,ploidy,gen1List,gen2List,vida_media):
        #quiza el nombre no sea necesario para nada
        self.spName = name
        self.spSize = size
        self.spPloidy = ploidy
        self.gene_1_List = gen1List
        self.gene_2_List = gen2List
        self.vida_media = vida_media
        
        self.sex = self.sex()
        self.ide = "id"+str(nom)
        self.age = 0
        self.genotype = self.firstGenotipo()
        
    def sex(self):
        if randint(0,1)==0:
            return "Male"
        else:
            return 'Female'
        
    def edad(self):
        #el usuario debera pasar algun parametro para indicar la dist por edades
        pass
    
    def firstGenotipo(self,homogeneous=False):
        if homogeneous:
            return {'gen1':'Aa','gen2':'Bb'}
        else:
            return self.chooseGenotype()
        
    def chooseGenotype(self):
        dict = {}

        dict['gen1']= random.choice(self.gene_1_List)
        dict['gen2']= random.choice(self.gene_2_List)
        
        return dict
    
#en un principio esto no es necesario ya que no interesa conocer a un individuo concreto
    def printer(self):
        print("este individuo es {}, su sexo es {} y su genotipo es {}".format(self.ide,self.sex,self.genotype))


### Problemas:

- Se ha eliminado la estructura de herencia, ahora un metodo (generateIndividuals) de la clase Population crea las nuevas instancias Individual, se les pasan los atributos de Population
- Se ha creado la funcion Population.genot() que genera las frecuencias genotipicas a partir de las frecuencias alelicas, hay que encontrar alguna forma de asignar un genotipo a cada individuo dependiendo de la frecuencia genotipica

In [41]:
#se crea una nueva poblacion donde se especifican caracteristicas generales de esta
    #size es el numero de individuos
    #name el nombre de la especie
    #ploidy es la ploidia de la poblacion (haploide=1,diploide=2)
    #vida media es la vida media
    #freq son las frecuencias alelicas en cada locus, es una tupla de diccionarios
    #mu es la tasa de mutacion de los alelos (de A a a y viceversa..)
humanos = Population(size=100,
                     name="Megadolon",
                     ploidy=2,
                     vida_media=23,
                     freq={'A':(0.4,0.6),'B':(0.6,0.4)})

In [42]:
#se generan individuos en esa poblacion
humanos.generateIndividuals()

se han generado un total de 100 individuos de la poblacion Megadolon


In [ ]:
#parametro opcional show, permite elegir cuantos elementos se muestran (por defecto se muestran 10)
humanos.printIndiv(show=15)

In [ ]:
humanos.getMeanAge()

In [ ]:
#devuelve un diccionario con un resumen de las frecuencias absolutas para cada gen
humanos.getGenotype()

In [ ]:
#comienza a correr el tiempo (no implementado)
#humanos.age()

In [ ]:
#check if population attribute gen1List is actually a object class parameter
print(hasattr(Population,'summary'))
print(hasattr(Population,'Population.size'))
print(hasattr(Individual,'vida_media'))
print(vars(Population))

In [46]:
humanos.genot()

{'A': [0, 0, 0], 'B': [0, 0, 0]}
